<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Chisel Demo
**Next: [Introduction to Scala](1_intro_to_scala)**

## Motivation
Perhaps you're an interested studen who heard the name "Chisel" tossed about, or maybe you're a seasoned hardware design veteran who has been tasked by your manager to explore Chisel as a new HDL alternative. Either way if you are new to Chisel, you want to figure out as fast as possible what all the fuss is about. Look no futher - let's see what Chisel has to offer!

## Setup
The following cell downloads the dependencies needed for Chisel. You will see it in all future notebooks. **Run this cell now**.

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

These statements are needed to import Chisel. **Run this cell now** before running any future code blocks.

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# Bringing the Power of Parameterizability and Abstraction to Digital Design
// What will you expect to learn?

<span style="color:blue">**Example: An FIR Filter**</span><br>
Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module` (TODO write more).

In [ ]:
// 3-point moving average implemented in the style of a FIR filter
class MovingAverage3(bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val z1 = RegNext(io.in)
  val z2 = RegNext(z1)

  io.out := (io.in * 1.U) + (z1 * 1.U) + (z2 * 1.U)
}

There's a lot here! The following explains how to think of each line in terms of the hardware we are describing.

```scala
class Passthrough extends Module {
```
We declare a new module called `Passthrough`. `Module` is a built-in Chisel class that all hardware modules must extend.

In [ ]:
visualize(() => new MovingAverage3(3))

In [ ]:
// Viewing the Verilog for debugging
println(getVerilog(new MovingAverage3(3)))

In [ ]:
// Viewing the firrtl for debugging
println(getFirrtl(new MovingAverage3(3)))

---
# You're done!

[Return to the top.](#top)